Step 1: Imports

In [17]:
## Basic module imports
import pandas as pd
import os
import numpy as np 
from PIL import Image, ImageDraw

## Maibi_cv imports
import lib.detection.transforms as transforms
import lib.detection.engine as engine

## Torch imports
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter

## Visualization imports
from IPython.display import display

## Torchvision imports 
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


Step 2: Create dictionary containing Images (key) and ImagePath (value)

In [13]:
def get_imagepathdf (root_path): 
    file_dict={}
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file.endswith((".jpg", ".jpeg")):
                file_dict[file]=os.path.abspath(os.path.join(root,file))
    dict_df = pd.DataFrame.from_dict(file_dict, orient='index', columns=['filepath'])
    dict_df.reset_index(inplace=True)
    dict_df.rename(columns={'index': 'filename'}, inplace=True)
    return dict_df

Step 3: Merge all CSV files with annotations in a single CSV file

In [14]:
def get_annotationsdf(root_path):
    dfs=[]
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file == "final.csv":
                df = pd.read_csv(os.path.join(root,file))
                dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df['nature_int']=combined_df['nature'].map(labelmapping)
    return combined_df  

Step 4: Check for broken images

In [15]:
def filter_rows(input_df): 
    for index, row in input_df.iterrows(): 
        try:
            img = Image.open(row['filepath'])
        except ValueError as e: 
            print('exception for: ' +  row['filename']) 

Step 5: Create DataSet

In [ ]:
RELPATH=".\\data"
FILENAME=".\\full_dataset.csv"

imagepath_df = get_imagepathdf(RELPATH)
annotations_df = get_annotationsdf(RELPATH)
merged_df = pd.merge(annotations_df, imagepath_df, left_on='image', right_on='filename', how='inner')
merged_df.to_csv(RELPATH+FILENAME, index=False)

filter_rows(merged_df)

absolute_path = os.path.abspath(RELPATH+FILENAME)

train_tfm = Trans.Compose([
    Trans.ToTensor(),  # converts the image, a PIL image, into a PyTorch Tensor
    #Trans.RandomHorizontalFlip(0.5)  # randomly flip the training images
])

val_tfm = Trans.ToTensor()

full_dataset = TowerDataset(absolute_path, train_tfm)

Step 6: Test DataSet

In [ ]:
# Get image based on index
img, target = full_dataset[0]
draw = ImageDraw.Draw(img)
for row in enumerate(target["boxes"]):
    draw.rectangle([row[1][0],row[1][1],row[1][2],row[1][3]], outline='red', width=6)
#display(img)

# Get image based on name for testing purposes 
img2, target2 = full_dataset.getitem_name('P12_H4-DW_DJI_0792.jpeg')
draw = ImageDraw.Draw(img2)
for row in enumerate(target2["boxes"]):
    draw.rectangle([row[1][0],row[1][1],row[1][2],row[1][3]], outline='red', width=6)
#display(img2)

Step 7: Split DataSet & Create DataLoaders

In [ ]:
dataset_size = len(full_dataset)

train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

data_loader_train = DataLoader(
    train_dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=collate_fn
)

data_loader_val = DataLoader(
    val_dataset, batch_size=2, shuffle=False, num_workers=4,
    collate_fn=collate_fn
)

Step 8: Create & Initialize the Model

In [ ]:
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# our dataset has two classes only - background and product
num_classes = 5

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = StepLR(optimizer,
                      step_size=3,
                      gamma=0.1)

Step 9: Start TensorBoard for Logging

In [ ]:
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# our dataset has two classes only - background and product
num_classes = 5

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = StepLR(optimizer,
                      step_size=3,
                      gamma=0.1)

C:\Users\dgdwi\AppData\Local\Temp\ipykernel_25664\1851517766.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(".\\data\\labels_corrected.csv")


,Unnamed: 0,pylone,degat,degat_IA,face,partie,partie_du_pylone,catalogue,localisation,symbol,...,altitude,topleftx,toplefty,bottomrightx,bottomrighty,path_to_image,3Dpos,label,checked,corrected
0,0,P100,1.0,1.0,H2,fût,fût,1.1 MEMBRURE/HOOFDSTIJL,01. membrure/ hoofdstijl,rect,...,0.0,3424.0,1708.0,3387.0,1617.0,/media/tom/Elements/rgb_dataset/Bressoux_70.35...,NaN,rouille2,0.0,0.0
1,1,P100,2.0,12.0,D,fût,fût,6. TREILLIS/ VAKWERKSTAAF,06. treillis/vakwerkstaaf,circle,...,0.0,2522.0,1447.0,2405.0,1361.0,/media/tom/Elements/rgb_dataset/Bressoux_70.35...,NaN,rouille3,1.0,0.0
2,2,P100,2.0,2.0,D,fût,fût,6. TREILLIS/ VAKWERKSTAAF,06. treillis/vakwerkstaaf,circle,...,0.0,2245.0,1400.0,2138.0,1269.0,/media/tom/Elements/rgb_dataset/Bressoux_70.35...,NaN,rouille3,1.0,0.0
3,3,P100,3.0,60.0,D,fût,fût,1.2. MEMBRURE-TREILLIS/ HOOFDSTIJL-VAKWERKSTAVEN,02. joint/verbinding,triangle,...,0.0,2277.0,3471.0,2093.0,3234.0,/media/tom/Elements/rgb_dataset/Bressoux_70.35...,NaN,rouille3,1.0,0.0
4,4,P100,3.0,3.0,D,fût,fût,1.2. MEMBRURE-TREILLIS/ HOOFDSTIJL-VAKWERKSTAVEN,02. joint/verbinding,triangle,...,0.0,2345.0,1523.0,2294.0,1388.0,/media/tom/Elements/rgb_dataset/Bressoux_70.35...,NaN,rouille3,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74214,74214,P7,39.0,63.0,H3,tête,consoles,8.1. BEC CONSOLE/ MASTARMUITEINDE,02. joint/verbinding,cross,...,0.0,4027.0,2359.0,3813.0,2066.0,/media/tom/Elements/rgb_dataset/Lendelede_70.5...,NaN,rouille2,1.0,0.0
74215,74215,P7,40.0,66.0,H2,fût,fût,1.1 MEMBRURE/HOOFDSTIJL,01. membrure/ hoofdstijl,rect,...,0.0,2369.0,1371.0,2312.0,1301.0,/media/tom/Elements/rgb_dataset/Lendelede_70.5...,NaN,rouille2,1.0,0.0
74216,74216,P7,41.0,68.0,H4,tête,chevalet,1.1 MEMBRURE/HOOFDSTIJL,01. membrure/ hoofdstijl,rect,...,0.0,2572.0,1475.0,2530.0,1389.0,/media/tom/Elements/rgb_dataset/Lendelede_70.5...,NaN,peinture1,1.0,0.0
74217,74217,P7,42.0,70.0,DI,tête,consoles,3.1. LONGERON/ LIGGER,07. longeron/ ligger,circle,...,0.0,1716.0,1742.0,1595.0,1625.0,/media/tom/Elements/rgb_dataset/Lendelede_70.5...,NaN,rouille2,1.0,0.0
